# Getting Started

## Prerequisites

### Required Packages for AFLOW

AFLOW requires the following commands to be in the path:

* `gcc` (Linux) or `clang++` (Mac)
* `make`
* `perl`
* `wget`
* `xz`

The packages that contain these commands need to be installed before installing AFLOW. On a Mac, `xcode` needs to be installed as well (available in the app store).

For this workshop, `curl` and `python3` are also needed. They are not required for AFLOW to function though.

While not required for the workshop, the following packages are needed for full AFLOW functionality:

* `convert` (part of `imagemagick`)
* `gnuplot`
* `gs` (GhostScript)
* `pdflatex` (part of `texlive`)

To test whether a package in needed, open a terminal and use the command `which package` (replace "package" with the desired package). If the output is empty, the command is not available.

### Installing Missing Packages on Mac

The following section describes how to install missing packages using the package manager `homebrew`. To install `homebrew`, open a terminal and execute the following command:

`/usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"`

To install a missing package, use the command `brew install package` (replace "package" with the desired package).

### Preparing the AFLOW Installation on Windows

AFLOW does not natively run on Windows. However, it is possible to run AFLOW on a Windows machine using emulators.

#### Ubuntu for Windows (Windows 10)

1. Install `Ubuntu for Windows` from the Microsoft Store and open it.
2. After the initial setup, run `sudo apt-get update`.
3. Then use `sudo apt-get install g++ make` to install the required packages. The other packages from the list above should already be installed.
4. Open `/etc/ImageMagick-6/policy.xml` and find the line `<policy domain=“coder” rights=“none” pattern=“PS” />`. Replace "none" with "read|write".
5. For this workshop, also install `python3-pip`. Python3 should already be installed.
6. While not required for the workshop, the following command should be run for full AFLOW functionality: `sudo apt-get install gnuplot imagemagick texlive`

#### Cygwin (Windows 7+)

1. Download Cygwin: https://cygwin.com/install.html
2. Run the installer until you get to the "Select Packages" screen.
3. Select the following packages and click "Next" to install
     * `curl`
     * `gcc-g++`
     * `make`
     * `perl`
     * `wget`
4. Also install a text editor such as `vim`, `emacs`, or `nano`.
5. To install jupyter, the following packages are needed:
     * `python3`
     * `python3-devel`
     * `python36-numpy`
     * `python36-pip`
     * `python3-zmq`
6. While not required for the workshop, the following packages are needed for full AFLOW functionality:
     * `ghostscript`
     * `gnuplot`
     * `imagemagick`
     * `texlive`

**Warning**: Installing jupyter inside a virtual environment may not work with Cygwin.

### Required Python Packages

The following packages are required for the exercises in this workshop:

* attrs>=18.2.0
* certifi>=2018.11.29
* chardet>=3.0.4
* decorator>=4.3.2
* idna>=2.8
* ipython-genutils>=0.2.0
* jsonschema>=3.0.0
* jupyter>=1.0
* nbformat>=4.4.0
* numpy>=1.16.2
* plotly>=3.6.1
* pyrsistent>=0.14.11
* pytz>=2018.9
* requests>=2.21.0
* retrying>=1.3.3
* six>=1.12.0
* traitlets>=4.3.2
* urllib3>=1.24.1

These packages will be installed automatically with this notebook.

## Installing AFLOW

Installing AFLOW is fully automatized and should not require user intervention. The setup will download a script called `xaflow` which downloads and compiles the newest AFLOW version automatically.

To change the location of the xaflow script and the AFLOW binary, set the variable `USER_ULB` to the desired path (default: "~/bin/").

In [ ]:
USER_ULB = ""

To change the location of the source code, set the variable `USER_AWD` to the desired path (default: "~/src/AFLOW/"):

In [ ]:
USER_AWD = ""

If all prerequisites are fulfilled, run this jupyter notebook. AFLOW should compile and install automatically. The shell and jupyter may need to be restarted for these changes to take effect.

### Code

The following is the code that is executed to install AFLOW and can be safely ignored.

In [ ]:
# Replace ~ with the full path of the home directory
import os

if '~' in USER_AWD:
    USER_AWD = USER_AWD.replace('~', os.path.expanduser('~'))
if USER_AWD and USER_AWD[-1] != '/':
    USER_AWD += '/'
    
if '~' in USER_ULB:
    USER_ULB = USER_ULB.replace('~', os.path.expanduser('~'))
if USER_ULB and USER_ULB[-1] != '/':
    USER_ULB += '/'

# Set default ULB for xaflow
DEFAULT_ULB = os.path.expanduser('~') + '/bin/'

In [ ]:
# Check whether the current operating system is supported
def checkOS():
    supported_platforms = ['linux', 'linux2', 'darwin', 'cygwin']
    import sys
    platform = sys.platform
    if platform not in supported_platforms:
        message = "Operating system " + platform + " not supported."
        if (platform == 'win32'):
            message += " Please use Ubuntu for Windows (Windows 10) or cygwin (Windows 8 or older)."
        raise RuntimeError(message)

In [ ]:
# Check whether all required commands are available
def checkRequiredPackages():
    import sys
    required_packages = ['convert', 'curl', 'make', 'perl', 'pip3', 'python', 'wget', 'xz']
    #                     'gnuplot', 'gs','pdflatex']
    
    # Mac requires clang++, linux gcc
    if (sys.platform == 'darwin'):
        required_packages.append('clang++')
    else:
        required_packages.append('gcc')
    missing_packages = []
    
    for package in required_packages:
        cmd = !which $package
        if not cmd:
            missing_packages.append(package)
        
    if len(missing_packages) > 0:
        message = "Could not find the following commands: " + ", ".join(missing_packages) + \
                   ". Please install the required packages."
        raise RuntimeError(message)

In [ ]:
# Download the xaflow script
def downloadXAflow():
    import os, sys
    try:
        import urllib.request as urlrequest
    except ImportError:
        import urllib as urlrequest
    
    # Create directory if neeeded
    ULB = USER_ULB
    if not ULB:
        ULB = DEFAULT_ULB
    if not os.path.isdir(ULB):
        os.mkdir(ULB)
        if not os.path.isdir(ULB):
            raise IOError("Could not create directory " + ULB + ".")

    # Add ULB to path
    pathvar = os.environ.get('PATH')
    if pathvar is not None:
        path = pathvar.split(':')
    else:
        path = []
    if ULB not in path:
        os.environ['PATH'] = ULB + ":" + os.environ['PATH']
    
    # Add to .bashrc if needed
    bashrc = os.path.expanduser('~')
    if (sys.platform == 'darwin'):
        bashrc += '/.bash_profile'
    else:
        bashrc += '/.bashrc'
    found = False
    export_string = "export PATH=" + ULB + ":$PATH"
    with open(bashrc, 'rt') as f:
        if export_string in f.read():
            found = True
    if not found:
        !echo "export PATH=$ULB:\$$PATH" >> $bashrc
    
    # Download and make executable
    xaflow = ULB + 'xaflow'
    if os.path.exists(xaflow):
        os.remove(xaflow)
    urlrequest.urlretrieve('http://materials.duke.edu/AFLOW/xaflow', xaflow)
    os.chmod(xaflow, 0o555)
    
    # Check
    cmd = !which xaflow
    if not cmd:
        message = "Could not find command xaflow. Please make sure that the file is present and executable."
        raise RuntimeError(message)

In [ ]:
# Compiles and installs AFLOW
def installAflow():
    if USER_AWD:
        !xaflow AWD=$USER_AWD
    else:
        !xaflow
    if USER_ULB:
        !xaflow install ULB=$USER_ULB
    else:
        !xaflow install
        
    cmd = !which aflow
    if not cmd:
        message = "Could not find command aflow. Installation unsuccessful."
        raise RuntimeError(message)

In [ ]:
def installPythonPackages():
    !pip3 install -r requirements.txt

In [ ]:
checkOS()
checkRequiredPackages()
installPythonPackages()
downloadXAflow()
installAflow()